In [1]:
import typing
import dataclasses
import logging

import pydantic

from seis_lab_data import schemas
from seis_lab_data.webapp import forms

logging.basicConfig(level=logging.DEBUG)

In [2]:
# a synthetic request class that we can use just for instantiating a starlette_wtf form
@dataclasses.dataclass
class FakeRequest:
    method: str = "POST"
    state: str = ""

In [3]:
r1 = FakeRequest()

In [4]:
f1 = forms.SurveyRelatedRecordCreateForm(r1)

In [10]:
for asset_form_field in f1.assets[0]:
    print(asset_form_field)
    print(asset_form_field.name)
    print(asset_form_field.short_name)
    print("---")

<table id="assets-0-name"><tr><th><label for="assets-0-name-en">English name</label></th><td><input id="assets-0-name-en" maxlength="100" name="assets-0-name-en" type="text" value=""></td></tr><tr><th><label for="assets-0-name-pt">Portuguese name</label></th><td><input id="assets-0-name-pt" maxlength="100" name="assets-0-name-pt" type="text" value=""></td></tr></table>
assets-0-name
name
---
<table id="assets-0-description"><tr><th><label for="assets-0-description-en">English description</label></th><td><textarea id="assets-0-description-en" maxlength="500" name="assets-0-description-en">
</textarea></td></tr><tr><th><label for="assets-0-description-pt">Portuguese description</label></th><td><textarea id="assets-0-description-pt" maxlength="500" name="assets-0-description-pt">
</textarea></td></tr></table>
assets-0-description
description
---
<input id="assets-0-relative_path" name="assets-0-relative_path" type="text" value="">
assets-0-relative_path
relative_path
---
<ul id="assets-0-

In [18]:
loc = ("assets", 0, "name")

In [19]:
asset_list = getattr(f1, loc[0])

In [20]:
asset_form = asset_list[0]

In [21]:
getattr(asset_form, loc[2])

'assets-0'

In [27]:
asset_form.short_name

'assets-0'

In [22]:
getattr(asset_form, "asset_name")

In [33]:
for f in asset_form:
    print(f"{f=}, {f.short_name=}")

f=<wtforms.fields.form.FormField object at 0x7156dd916420>, f.short_name='name'
f=<wtforms.fields.form.FormField object at 0x7156dd916510>, f.short_name='description'
f=<wtforms.fields.simple.StringField object at 0x7156dd9165a0>, f.short_name='relative_path'
f=<wtforms.fields.list.FieldList object at 0x7156dd916630>, f.short_name='links'


In [4]:
f1 = forms.ProjectCreateForm(
    r1,
    data={
        "name": {
            "pt": "",
            "en": "",
        },
        "description": {
            "pt": "",
            "en": "",
        },
    },
)
f2 = forms.ProjectCreateForm(r1)

In [5]:
await f1.validate_on_submit()
await f2.validate_on_submit()

True

In [6]:
validated_form = forms.validate_form_with_model(f2, schemas.ProjectCreate)

ERROR:seis_lab_data.webapp.forms:pydantic errors exc.errors()=[{'type': 'missing', 'loc': ('id',), 'msg': 'Field required', 'input': {'name': {'en': None, 'pt': None}, 'description_en': None, 'description_pt': None, 'root_path': None, 'links': []}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}, {'type': 'missing', 'loc': ('owner',), 'msg': 'Field required', 'input': {'name': {'en': None, 'pt': None}, 'description_en': None, 'description_pt': None, 'root_path': None, 'links': []}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}, {'type': 'string_type', 'loc': ('name', 'en'), 'msg': 'Input should be a valid string', 'input': None, 'url': 'https://errors.pydantic.dev/2.11/v/string_type'}, {'type': 'missing', 'loc': ('description',), 'msg': 'Field required', 'input': {'name': {'en': None, 'pt': None}, 'description_en': None, 'description_pt': None, 'root_path': None, 'links': []}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}, {'type': 'string_type', 'loc': ('root_path',

In [7]:
validated_form.errors

{'name': {'en': ['Input should be a valid string']},
 'root_path': ['Input should be a valid string']}

In [8]:
validated_form = forms.validate_form_with_model(f1, schemas.ProjectCreate)

ERROR:seis_lab_data.webapp.forms:pydantic errors exc.errors()=[{'type': 'missing', 'loc': ('id',), 'msg': 'Field required', 'input': {'name': {'en': '', 'pt': ''}, 'description_en': None, 'description_pt': None, 'root_path': None, 'links': []}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}, {'type': 'missing', 'loc': ('owner',), 'msg': 'Field required', 'input': {'name': {'en': '', 'pt': ''}, 'description_en': None, 'description_pt': None, 'root_path': None, 'links': []}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}, {'type': 'string_too_short', 'loc': ('name', 'en'), 'msg': 'String should have at least 1 character', 'input': '', 'ctx': {'min_length': 1}, 'url': 'https://errors.pydantic.dev/2.11/v/string_too_short'}, {'type': 'missing', 'loc': ('description',), 'msg': 'Field required', 'input': {'name': {'en': '', 'pt': ''}, 'description_en': None, 'description_pt': None, 'root_path': None, 'links': []}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}, {'type': 'str

In [9]:
validated_form.errors

{'name': {'en': ['String should have at least 1 character']},
 'root_path': ['Input should be a valid string']}

In [10]:
validated_form.name.errors

{}

In [ ]:
schemas.ProjectCreate.model_fields

In [19]:
def get_all_field_locs(model_class: pydantic.BaseModel) -> list[tuple]:
    """
    Generates a list of all possible field locations (loc tuples) for a Pydantic model.

    This function recursively inspects the model's fields, including nested
    BaseModels, to build a list of all possible paths where a validation
    error could occur.

    Args:
        model_class: The Pydantic BaseModel class to inspect.

    Returns:
        A list of tuples, where each tuple is a potential 'loc' for a validation error.
    """
    locs = []

    def _traverse_fields(current_model, current_path):
        """Helper function to recursively traverse the model's fields."""
        for field_name, field_info in current_model.model_fields.items():
            # Create a new path by extending the current path with the field name
            new_path = current_path + (field_name,)
            locs.append(new_path)

            # Check if the field's annotation is a Pydantic BaseModel.
            # We use `isinstance` on the annotation's type to handle generics like List[Address]
            annotation = field_info.annotation
            if hasattr(annotation, "__origin__") and annotation.__origin__ is list:
                # Handle lists of models
                inner_type = annotation.__args__[0]
                if issubclass(inner_type, pydantic.BaseModel):
                    # We can't know the list index beforehand, so we represent it with an ellipsis
                    list_path = new_path + (...,)
                    _traverse_fields(inner_type, list_path)
            elif issubclass(annotation, pydantic.BaseModel):
                # Recursively traverse the nested model
                _traverse_fields(annotation, new_path)

    _traverse_fields(model_class, ())
    return locs

In [20]:
get_all_field_locs(schemas.ProjectCreate)

[('id',),
 ('owner',),
 ('name',),
 ('description',),
 ('root_path',),
 ('links',),
 ('links', Ellipsis, 'url'),
 ('links', Ellipsis, 'media_type'),
 ('links', Ellipsis, 'relation'),
 ('links', Ellipsis, 'description')]

In [29]:
def get_all_field_locs(model_class: pydantic.BaseModel) -> list[tuple[typing.Any, ...]]:
    """
    Generates a list of all possible field locations (loc tuples) for a Pydantic model.

    This function recursively inspects the model's fields, including nested
    BaseModels and dictionaries, to build a list of all possible paths where a
    validation error could occur.

    Args:
        model_class: The Pydantic BaseModel class to inspect.

    Returns:
        A list of tuples, where each tuple is a potential 'loc' for a validation error.
    """
    locs = []

    def _traverse_fields(current_model, current_path):
        """Helper function to recursively traverse the model's fields."""
        for field_name, field_info in current_model.model_fields.items():
            # Create a new path by extending the current path with the field name
            new_path = current_path + (field_name,)
            locs.append(new_path)

            annotation = field_info.annotation
            origin = typing.get_origin(annotation)

            # Check if the field is a nested Pydantic model
            if isinstance(annotation, type) and issubclass(
                annotation, pydantic.BaseModel
            ):
                # Recursively traverse the nested model
                _traverse_fields(annotation, new_path)

            # Check for generic container types (like List or Dict)
            if origin:
                # Get the type of the elements inside the container
                args = typing.get_args(annotation)
                if args:
                    inner_type = args[0]
                    # For a Dict, the value type is the second argument
                    if origin is dict:
                        inner_type = args[1]

                    # If the inner type is a BaseModel, recurse on it
                    if isinstance(inner_type, type) and issubclass(
                        inner_type, pydantic.BaseModel
                    ):
                        # Use an ellipsis to represent an unknown index or key
                        container_path = new_path + (...,)
                        _traverse_fields(inner_type, container_path)

    _traverse_fields(model_class, ())
    return locs

In [30]:
get_all_field_locs(schemas.ProjectCreate)

[('id',),
 ('owner',),
 ('name',),
 ('description',),
 ('root_path',),
 ('links',),
 ('links', Ellipsis, 'url'),
 ('links', Ellipsis, 'media_type'),
 ('links', Ellipsis, 'relation'),
 ('links', Ellipsis, 'description')]

In [33]:
def get_all_field_locs(model_class: pydantic.BaseModel) -> list[tuple[typing.Any, ...]]:
    """
    Generates a list of all possible field locations (loc tuples) for a Pydantic model.

    This function recursively inspects the model's fields, including nested
    BaseModels, dictionaries, and lists, to build a list of all possible paths
    where a validation error could occur. It correctly handles Annotated types.

    Args:
        model_class: The Pydantic BaseModel class to inspect.

    Returns:
        A list of tuples, where each tuple is a potential 'loc' for a validation error.
    """
    locs = []

    def _traverse_fields(current_model, current_path):
        """Helper function to recursively traverse the model's fields."""
        for field_name, field_info in current_model.model_fields.items():
            # Get the effective annotation, unwrapping Annotated types
            annotation = field_info.annotation
            if typing.get_origin(annotation) is typing.Annotated:
                annotation = typing.get_args(annotation)[0]

            # Create a new path by extending the current path with the field name
            new_path = current_path + (field_name,)
            locs.append(new_path)

            origin = typing.get_origin(annotation)

            # Check if the field is a nested Pydantic model
            if isinstance(annotation, type) and issubclass(
                annotation, pydantic.BaseModel
            ):
                # Recursively traverse the nested model
                _traverse_fields(annotation, new_path)

            # Check for generic container types (like List or Dict)
            if origin:
                args = typing.get_args(annotation)
                if args:
                    inner_type = args[0]
                    # For a Dict, the value type is the second argument
                    if origin is dict:
                        inner_type = args[1]

                    # If the inner type is a BaseModel, recurse on it
                    if isinstance(inner_type, type) and issubclass(
                        inner_type, pydantic.BaseModel
                    ):
                        # Use an ellipsis to represent an unknown index or key
                        container_path = new_path + (...,)
                        _traverse_fields(inner_type, container_path)

    _traverse_fields(model_class, ())
    return locs

In [34]:
get_all_field_locs(schemas.ProjectCreate)

[('id',),
 ('owner',),
 ('name',),
 ('description',),
 ('root_path',),
 ('links',),
 ('links', Ellipsis, 'url'),
 ('links', Ellipsis, 'media_type'),
 ('links', Ellipsis, 'relation'),
 ('links', Ellipsis, 'description')]

In [38]:
schemas.SurveyRelatedRecordCreate(assets=[{"name": {}}])

ValidationError: 13 validation errors for SurveyRelatedRecordCreate
id
  Field required [type=missing, input_value={'assets': [{'name': {}}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
owner
  Field required [type=missing, input_value={'assets': [{'name': {}}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
name
  Field required [type=missing, input_value={'assets': [{'name': {}}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
description
  Field required [type=missing, input_value={'assets': [{'name': {}}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
survey_mission_id
  Field required [type=missing, input_value={'assets': [{'name': {}}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
dataset_category_id
  Field required [type=missing, input_